# 1. Exploration des Données - Olist E-commerce

Ce notebook explore les données Olist pour comprendre le comportement d'achat des clients.

**Objectifs :**
- Charger et examiner les datasets
- Analyser les distributions et tendances
- Identifier les variables clés pour la segmentation RFM

**Auteur :** Thomas Mebarki  
**Date :** Janvier 2026

## 1.1 Configuration et imports

In [ ]:
import sys
from pathlib import Path

# Ajouter le répertoire parent au path pour importer src
sys.path.insert(0, str(Path.cwd().parent))

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Configuration
pd.options.display.max_columns = 50
pd.options.display.float_format = '{:.2f}'.format
sns.set_style('whitegrid')
%matplotlib inline

print("Configuration terminée")

## 1.2 Chargement des données brutes

In [ ]:
from src.config import DATA_DIR

# Charger les datasets Olist originaux
data_path = DATA_DIR

customers_df = pd.read_csv(data_path / 'olist_customers_dataset.csv')
orders_df = pd.read_csv(data_path / 'olist_orders_dataset.csv')
items_df = pd.read_csv(data_path / 'olist_order_items_dataset.csv')
payments_df = pd.read_csv(data_path / 'olist_order_payments_dataset.csv')
reviews_df = pd.read_csv(data_path / 'olist_order_reviews_dataset.csv')
products_df = pd.read_csv(data_path / 'olist_products_dataset.csv')
sellers_df = pd.read_csv(data_path / 'olist_sellers_dataset.csv')

print(f"Customers: {customers_df.shape}")
print(f"Orders: {orders_df.shape}")
print(f"Items: {items_df.shape}")
print(f"Payments: {payments_df.shape}")
print(f"Reviews: {reviews_df.shape}")

## 1.3 Aperçu des datasets

In [ ]:
print("=== CUSTOMERS ===")
display(customers_df.head())
print(f"\nClients uniques: {customers_df['customer_unique_id'].nunique()}")

In [ ]:
print("=== ORDERS ===")
display(orders_df.head())
print(f"\nCommandes totales: {orders_df.shape[0]}")
print(f"Statuts: {orders_df['order_status'].value_counts().to_dict()}")

In [ ]:
print("=== ORDER ITEMS ===")
display(items_df.head())
print(f"\nPrix moyen: {items_df['price'].mean():.2f} BRL")
print(f"Panier moyen: {items_df.groupby('order_id')['price'].sum().mean():.2f} BRL")

## 1.4 Fusion des données pour l'analyse

In [ ]:
# Filtrer uniquement les commandes livrées
delivered_orders = orders_df[orders_df['order_status'] == 'delivered'].copy()
print(f"Commandes livrées: {len(delivered_orders)} ({len(delivered_orders)/len(orders_df)*100:.1f}%)")

# Fusionner avec les clients
data = delivered_orders.merge(customers_df, on='customer_id', how='left')

# Ajouter les montants (somme par commande)
order_amounts = items_df.groupby('order_id')['price'].sum().reset_index()
data = data.merge(order_amounts, on='order_id', how='left')

# Conversion des dates
data['order_purchase_timestamp'] = pd.to_datetime(data['order_purchase_timestamp'])

print(f"\nDataset fusionné: {data.shape}")
data.head()

## 1.5 Analyse temporelle des commandes

In [ ]:
# Plage temporelle
print(f"Période: {data['order_purchase_timestamp'].min()} à {data['order_purchase_timestamp'].max()}")

# Distribution par mois
data['order_month'] = data['order_purchase_timestamp'].dt.to_period('M')

fig, ax = plt.subplots(figsize=(14, 5))
orders_by_month = data.groupby('order_month').size()
orders_by_month.plot(kind='bar', ax=ax, color='steelblue', edgecolor='black')
ax.set_title('Évolution des commandes par mois', fontsize=14)
ax.set_xlabel('Mois')
ax.set_ylabel('Nombre de commandes')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

## 1.6 Analyse des clients

In [ ]:
# Statistiques par client unique
customer_stats = data.groupby('customer_unique_id').agg({
    'order_id': 'nunique',
    'price': 'sum',
    'order_purchase_timestamp': ['min', 'max']
}).round(2)

customer_stats.columns = ['nb_commandes', 'montant_total', 'premiere_commande', 'derniere_commande']

print(f"Nombre de clients uniques: {len(customer_stats)}")
print(f"\nStatistiques:")
customer_stats[['nb_commandes', 'montant_total']].describe()

In [ ]:
# Distribution du nombre de commandes par client
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Nombre de commandes
order_counts = customer_stats['nb_commandes'].value_counts().sort_index()
axes[0].bar(order_counts.index[:10], order_counts.values[:10], color='steelblue', edgecolor='black')
axes[0].set_title('Distribution du nombre de commandes par client', fontsize=12)
axes[0].set_xlabel('Nombre de commandes')
axes[0].set_ylabel('Nombre de clients')

# Montant total
axes[1].hist(customer_stats['montant_total'], bins=50, color='green', edgecolor='black', alpha=0.7)
axes[1].set_title('Distribution du montant total par client', fontsize=12)
axes[1].set_xlabel('Montant total (BRL)')
axes[1].set_ylabel('Nombre de clients')
axes[1].set_xlim(0, 2000)  # Limiter pour la lisibilité

plt.tight_layout()
plt.show()

# Statistique clé
one_time_buyers = (customer_stats['nb_commandes'] == 1).sum()
print(f"\nClients avec 1 seule commande: {one_time_buyers} ({one_time_buyers/len(customer_stats)*100:.1f}%)")

## 1.7 Variables clés pour la segmentation RFM

Les variables identifiées pour la segmentation :

| Variable | Description | Source |
|----------|-------------|--------|
| **Recency** | Jours depuis le dernier achat | `order_purchase_timestamp` |
| **Frequency** | Nombre de commandes | `order_id` (count unique) |
| **Monetary** | Montant total dépensé | `price` (sum) |

In [ ]:
# Vérification des données nécessaires
print("Colonnes disponibles pour RFM:")
print(f"  - customer_unique_id: {data['customer_unique_id'].nunique()} valeurs uniques")
print(f"  - order_purchase_timestamp: {data['order_purchase_timestamp'].notna().sum()} valeurs non-nulles")
print(f"  - price: {data['price'].notna().sum()} valeurs non-nulles")
print(f"  - order_id: {data['order_id'].nunique()} commandes uniques")

# Valeurs manquantes
print(f"\nValeurs manquantes:")
print(data[['customer_unique_id', 'order_purchase_timestamp', 'price']].isna().sum())

## 1.8 Sauvegarde des données préparées

In [ ]:
from src.config import RAW_DATA_DIR

# Sélectionner les colonnes nécessaires pour le calcul RFM
rfm_input_data = data[['customer_unique_id', 'order_id', 'order_purchase_timestamp', 'price']].copy()

# Sauvegarder pour le prochain notebook
output_path = RAW_DATA_DIR / 'transactions_clean.csv'
rfm_input_data.to_csv(output_path, index=False)

print(f"Données sauvegardées: {output_path}")
print(f"Shape: {rfm_input_data.shape}")

## 1.9 Conclusion

**Points clés de l'exploration :**

1. **Volume** : ~96K clients uniques, ~96K commandes livrées
2. **Comportement** : ~97% des clients n'ont qu'une seule commande (faible rétention)
3. **Valeur** : Panier moyen ~137 BRL
4. **Période** : Données de septembre 2016 à août 2018

**Prochaine étape :** Feature Engineering RFM dans le notebook 02.